## Adjusted Plus-Minus

Source: https://github.com/TravisRPetersen/sports-analytics-spring18/blob/master/lesson_06/Count_The_Basket_Calculating%20Adjusted%20Plus_Minus.pdf

In [34]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

## Import Play by Play Data

In [2]:
matchups0708 = pd.read_csv('matchups20072008reg20081211.txt', sep="\t")

In [3]:
matchups0708.head()

,GameID,StartTime,EndTime,ElapsedTime,ElapsedSecs,HomePlayer1ID,HomePlayer2ID,HomePlayer3ID,HomePlayer4ID,HomePlayer5ID,...,OverallRtgHomevsAway,OverallRtgAwayvsHome,OffensiveReboundsHome,OffensiveReboundsAway,DefensiveReboundsHome,DefensiveReboundsAway,OffensiveReboundingRateHome,OffensiveReboundingRateAway,DefensiveReboundingRateHome,DefensiveReboundingRateAway
0,20071030HOULAL,00:00:01,00:00:00,00:00:01,1,189,186,103,516,459,...,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
1,20071030HOULAL,00:00:02,00:00:01,00:00:01,1,189,186,575,103,459,...,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
2,20071030HOULAL,00:01:32,00:00:02,00:01:30,90,189,186,575,103,459,...,165.0000,-165.0000,0,0,1,0,NaN,0.0,1.0,NaN
3,20071030HOULAL,00:01:36,00:01:32,00:00:04,4,189,186,103,664,459,...,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN
4,20071030HOULAL,00:04:30,00:01:36,00:02:54,174,189,186,103,516,459,...,-86.6667,86.6667,0,0,1,3,0.0,0.0,1.0,1.0


In [4]:
matchups0708[['PointsScoredHome','PointsScoredAway','HomePlayer1ID']].head()

,PointsScoredHome,PointsScoredAway,HomePlayer1ID
0,0,0,189
1,1,0,189
2,9,3,189
3,3,0,189
4,4,10,189


## Add Player ID Lists

In [6]:
matchups0708['HomePlayers'] = matchups0708.apply(lambda row: ['P'+str(row['HomePlayer1ID']), 'P'+str(row['HomePlayer2ID']), 
                                                             'P'+str(row['HomePlayer3ID']), 'P'+str(row['HomePlayer4ID']), 
                                                            'P'+str(row['HomePlayer5ID'])], axis=1)
matchups0708['AwayPlayers'] = matchups0708.apply(lambda row: ['P'+str(row['AwayPlayer1ID']), 'P'+str(row['AwayPlayer2ID']), 
                                                             'P'+str(row['AwayPlayer3ID']), 'P'+str(row['AwayPlayer4ID']), 
                                                            'P'+str(row['AwayPlayer5ID'])], axis=1)

In [7]:
matchups0708[['PointsScoredHome','PointsScoredAway','HomePlayers','AwayPlayers']].head()

,PointsScoredHome,PointsScoredAway,HomePlayers,AwayPlayers
0,0,0,"[P189, P186, P103, P516, P459]","[P308, P318, P305, P302, P62]"
1,1,0,"[P189, P186, P575, P103, P459]","[P308, P198, P508, P318, P305]"
2,9,3,"[P189, P186, P575, P103, P459]","[P308, P198, P318, P305, P302]"
3,3,0,"[P189, P186, P103, P664, P459]","[P308, P198, P318, P305, P302]"
4,4,10,"[P189, P186, P103, P516, P459]","[P308, P198, P508, P305, P302]"


In [8]:
matchups0708['PointsScoredperPoss_Home'] = matchups0708.apply(lambda row: 125002/114854 if row['PossessionsHome'] ==0 else row['PointsScoredHome']/row['PossessionsHome'], axis=1)
matchups0708['PointsScoredperPoss_Away'] = matchups0708.apply(lambda row: 120811/114896 if row['PossessionsAway'] ==0 else row['PointsScoredAway']/row['PossessionsAway'], axis=1)

In [9]:
matchups0708['MARGIN'] = matchups0708.apply(lambda row: 100*(row['PointsScoredperPoss_Home'] - row['PointsScoredperPoss_Away']), axis=1)

In [10]:
matchups0708[['MARGIN','HomePlayers','AwayPlayers']].head()

,MARGIN,HomePlayers,AwayPlayers
0,3.687431,"[P189, P186, P103, P516, P459]","[P308, P318, P305, P302, P62]"
1,3.687431,"[P189, P186, P575, P103, P459]","[P308, P198, P508, P318, P305]"
2,165.000000,"[P189, P186, P575, P103, P459]","[P308, P198, P318, P305, P302]"
3,194.851866,"[P189, P186, P103, P664, P459]","[P308, P198, P318, P305, P302]"
4,-86.666667,"[P189, P186, P103, P516, P459]","[P308, P198, P508, P305, P302]"


## Import Players Data

In [13]:
players0708 = pd.read_csv('playerstats20072008reg20081211.txt', sep="\t")

In [14]:
players0708.head()

,PlayerID,PlayerTrueName,PlayerTeams,SimpleMin,SimplePossFor,SimplePossOpp,SimplePointsFor,SimplePointsOpp,SimpleOffRtg,SimpleDefRtg,...,WeightedORebRateOffCourt,WeightedDRebRateOffCourt,WeightedOverallRtgOnCourt,WeightedORebRateOnCourt,WeightedDRebRateOnCourt,WeightedOverallRtgOnCourtMinusOffCourt,WeightedORebRateOnCourtMinusOffCourt,WeightedDRebRateOnCourtMinusOffCourt,AdjustedPM,AdjustedPMStdErr
0,1,"Howard, Josh",DAL,2764.4000,5229,5210,5869,5458,112.239,104.760,...,0.270300,0.730600,7.47936,0.294488,0.726077,8.11890,0.024200,-0.004500,4.15536,4.30156
1,2,"Thomas, Kurt","SAS, SEA",1584.2167,3046,3047,3095,3167,101.609,103.938,...,0.291066,0.717080,-2.32963,0.268542,0.733083,2.96655,-0.023291,0.017352,3.20933,3.93265
2,3,"Dampier, Erick",DAL,1755.2167,3274,3272,3651,3483,111.515,106.449,...,0.273700,0.720300,5.06631,0.304294,0.736055,0.02480,0.030600,0.015800,-8.60147,4.82898
3,4,"Marion, Shawn","MIA, PHX",2315.1667,4659,4683,5173,4955,111.032,105.808,...,0.249395,0.685298,5.22417,0.237896,0.694397,5.48555,-0.011459,0.009827,2.71710,3.62157
4,5,"Nowitzki, Dirk",DAL,2768.8500,5249,5217,5982,5499,113.965,105.405,...,0.288600,0.726100,8.55916,0.286677,0.727938,11.87170,-0.001900,0.001800,7.93928,4.61853


In [15]:
players0708['PID'] = players0708['PlayerID'].apply(lambda x: 'P'+str(x))

In [16]:
players0708 = players0708[players0708['AdjustedPM'].notnull()].reset_index(drop=True)

In [17]:
player_ids_list = players0708['PID'].unique().tolist()

## Add Player Dummy Columns

In [18]:
for col in player_ids_list:
    matchups0708[col] = np.nan

In [20]:
x = 0
for col in player_ids_list:
    x+=1
    print(x)
    matchups0708[col] = matchups0708.apply(lambda row: 1 if col in row['HomePlayers'] else -1 if col in row['AwayPlayers'] else 0, axis=1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
matchups0708["Poss"] = matchups0708['PossessionsHome'] + matchups0708['PossessionsAway']

In [22]:
adj0708 = matchups0708[['MARGIN','Poss']+player_ids_list].copy()

In [23]:
#adj0708 = pd.read_csv('adj0708.csv')

In [24]:
adj0708.head()

,MARGIN,Poss,P1,P2,P3,P4,P5,P6,P7,P8,...,P708,P709,P711,P712,P713,P714,P715,P716,P719,P722
0,3.687431,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.687431,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,165.000000,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,194.851866,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-86.666667,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
("+").join(player_ids_list)

'P1+P2+P3+P4+P5+P6+P7+P8+P10+P12+P14+P15+P16+P17+P19+P20+P21+P22+P23+P24+P25+P26+P28+P29+P30+P31+P32+P33+P35+P36+P37+P39+P40+P42+P45+P46+P47+P48+P49+P52+P54+P55+P56+P59+P60+P62+P63+P65+P67+P68+P69+P70+P71+P75+P76+P77+P78+P79+P80+P83+P85+P86+P87+P88+P89+P90+P91+P92+P93+P94+P95+P96+P97+P98+P100+P101+P103+P109+P110+P111+P112+P113+P114+P115+P116+P117+P118+P119+P121+P122+P123+P125+P126+P128+P130+P132+P133+P136+P137+P138+P139+P140+P141+P143+P145+P147+P148+P149+P150+P151+P152+P154+P155+P158+P159+P163+P164+P165+P166+P168+P169+P170+P171+P172+P173+P174+P176+P180+P181+P185+P186+P187+P189+P190+P191+P193+P194+P196+P197+P198+P200+P201+P202+P203+P204+P205+P206+P209+P210+P211+P214+P215+P218+P220+P221+P222+P223+P224+P226+P231+P235+P236+P237+P239+P240+P241+P243+P244+P245+P246+P247+P249+P252+P253+P254+P255+P257+P258+P260+P261+P264+P265+P270+P271+P272+P273+P274+P275+P276+P277+P278+P279+P280+P281+P283+P284+P285+P287+P288+P289+P292+P293+P295+P296+P297+P298+P302+P305+P306+P307+P308+P311+P312+P313+P315+P316+P

In [27]:
apm_reg = smf.ols(formula='MARGIN ~ '+("+").join(player_ids_list), data=adj0708).fit()

In [28]:
apm_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 MARGIN   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.153
Date:                Mon, 19 Feb 2018   Prob (F-statistic):           2.81e-30
Time:                        19:37:55   Log-Likelihood:            -2.1582e+05
No. Observations:               35458   AIC:                         4.323e+05
Df Residuals:                   35118   BIC:                         4.352e+05
Df Model:                         339                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9303      0.587      8.398      0.000       3.780       6.081
P1             9.1047      4.367      2.085      0.037       0.546      17.664
P2             4.7565      4.558      1.043      0.297      -4.178      13.691
P3            -0.7537      5.043     -0.149      0.881     -10.638       9.130
P4             6.6796      4.365      1.530      0.126      -1.876      15.235
P5            14.7287      4.735      3.111      0.002       5.448      24.009
P6            10.1199      4.399      2.300      0.021       1.497      18.743
P7             5.8785      4.661      1.261      0.207      -3.257      15.014
P8            18.6883      5.315      3.516      0.000       8.270      29.106
P10            3.5591      5.187      0.686      0.493      -6.608      13.726
P12            0.5959      4.788      0.124      0.901      -8.789       9.981
P14            3.5795      4.229      0.846      0.397      -4.709      11.868
P15            8.6483      4.446      1.945      0.052      -0.066      17.362
P16            7.9637      4.900      1.625      0.104      -1.640      17.567
P17           -0.1769      4.756     -0.037      0.970      -9.499       9.145
P19            2.3224      5.845      0.397      0.691      -9.133      13.778
P20           11.6273      5.793      2.007      0.045       0.272      22.982
P21            7.2760      4.805      1.514      0.130      -2.141      16.693
P22            6.9613      5.219      1.334      0.182      -3.268      17.191
P23            6.3175      5.273      1.198      0.231      -4.017      16.652
P24           18.9083      4.292      4.406      0.000      10.496      27.320
P25            2.7893      4.786      0.583      0.560      -6.591      12.170
P26           13.5857      5.490      2.475      0.013       2.825      24.346
P28            0.3216      4.824      0.067      0.947      -9.134       9.777
P29            5.4231      4.307      1.259      0.208      -3.020      13.866
P30            9.6807      5.979      1.619      0.105      -2.039      21.400
P31           -4.8005      5.261     -0.912      0.362     -15.113       5.512
P32           -0.9136      6.039     -0.151      0.880     -12.749      10.922
P33            2.7099      6.041      0.449      0.654      -9.132      14.551
P35           -4.2446      6.484     -0.655      0.513     -16.953       8.464
P36           15.5089      4.887      3.173      0.002       5.930      25.088
P37            1.7585      4.894      0.359      0.719      -7.835      11.352
P39           -5.4807      6.739     -0.813      0.416     -18.688       7.727
P40           -2.4744      4.556     -0.543      0.587     -11.405       6.456
P42           -4.7705      5.385     -0.886      0.376     -15.325       5.784
P45            7.2538      5.146      1.409      0.159      -2.833      17.341
P4

In [29]:
apm_reg_params = apm_reg.params[1:].reset_index()

In [30]:
apm_reg_params.columns = ['PID','apm']

In [31]:
apm_reg_params.head()

,PID,apm
0,P1,9.104684
1,P2,4.756542
2,P3,-0.753723
3,P4,6.679631
4,P5,14.728657


In [32]:
apm_reg_params['Name'] = pd.merge(apm_reg_params, players0708, on='PID', how='left')['PlayerTrueName']

In [33]:
apm_reg_params.sort_values('apm', ascending=False).head(10)

,PID,apm,Name
215,P316,25.335203,"Jamison, Antawn"
217,P319,21.904883,"Bosh, Chris"
164,P240,20.999690,"Pierce, Paul"
19,P24,18.908287,"Ginobili, Manu"
7,P8,18.688324,"Nash, Steve"
178,P260,18.604267,"James, LeBron"
186,P274,18.219582,"Wallace, Rasheed"
313,P681,18.061160,"Young, Thaddeus"
130,P186,17.893549,"Bryant, Kobe"
61,P86,17.369972,"Johnson, Joe"
